In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
url= "https://www.stonefly.it/?" 

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

In [3]:
response = requests.get(url)
print(response)

<Response [200]>


In [4]:
#carichiamo tutte le dècolletè
nomi = []
prezzi_scontati = []
categorie = []
brands = []
prezzi_iniziali = []
taglie = []

url = "https://www.stonefly.it/collections/decollete-donna"

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

categoria_tag = soup.find("h1",class_="hyphenate")
categoria = categoria_tag.text.strip() if categoria_tag else "N/A"

brand_tag = soup.find("img", class_="header__logo-image")
src = brand_tag.get("src", "")
brand = src.split("/")[-1].split(".")[0].replace("Logo", "")


# Cerca le taglie nei filtri della pagina
taglie_divs = soup.find_all("label")  # o "input", "div" - dipende dalla struttura HTML
taglie_pagina = []

for div in taglie_divs:
    testo = div.get_text().strip()
    # Cerca numeri che sembrano taglie (es: 36, 37, 38.5, etc.)
    if re.match(r'^\d+(\.\d+)?$', testo):
        try:
            taglia = float(testo)
            if 30 <= taglia <= 50:  # Range ragionevole per scarpe donna
                taglie_pagina.append(taglia)
        except:
            pass

# Rimuovi duplicati
taglie_pagina = list(set(taglie_pagina))
taglie_numeriche = taglie_pagina if taglie_pagina else []

if taglie_numeriche:
    taglia_min = min(taglie_numeriche)
    taglia_max = max(taglie_numeriche)
else:
    taglia_min = None
    taglia_max = None

prodotti = soup.find_all('div', class_='each-product-gtm')

for prodotto in prodotti:
    nome = prodotto.find("a", class_="bold")
    nome = nome.text
    nomi.append(nome)
    prezzo_scontato = prodotto.find("sale-price", class_="text-on-sale")
    prezzo_scontato = prezzo_scontato.text
    prezzo_scontato = prezzo_scontato.replace("Prezzo scontato", "").replace("€", "").strip()
    prezzo_scontato = prezzo_scontato.replace(",", ".")
    prezzo_scontato = float(prezzo_scontato)
    prezzi_scontati.append(prezzo_scontato)

    prezzo_iniziale = prodotto.find("compare-at-price", class_="text-subdued line-through")
    prezzo_iniziale = prezzo_iniziale.text
    prezzo_iniziale = prezzo_iniziale.replace("Prezzo", "").replace("€", "").strip()
    prezzo_iniziale = prezzo_iniziale.replace(",", ".")
    prezzo_iniziale = float(prezzo_iniziale)
    prezzi_iniziali.append(prezzo_iniziale)

#prezzo_iniziale = prezzo_iniziale.replace("Prezzo scontato", "").replace("Prezzo", "").replace("€", "").strip()
#prezzo_iniziale = prezzo_iniziale.replace(",", ".")
#prezzo_iniziale = float(prezzo_iniziale)
    
    
    categorie.append(categoria)
    brands.append(brand)
    taglie.append((taglia_min, taglia_max))

df_dec = pd.DataFrame({
    "nome": nomi,
    "prezzo_iniziale": prezzi_iniziali,
    "prezzo_scontato": prezzi_scontati,
    "categoria": categorie,
    "brand": brands,
    "taglia_min": [t[0] for t in taglie],
    "taglia_max": [t[1] for t in taglie]
})

df_dec

,nome,prezzo_iniziale,prezzo_scontato,categoria,brand,taglia_min,taglia_max
0,SCARPE CON TACCO IRIS 1 NERO,119.99,71.99,Decollete donna,Stonefly,35.0,41.0
1,SCARPE CON TACCO IRIS 1 ARGENTO,119.99,71.99,Decollete donna,Stonefly,35.0,41.0
2,SCARPE DECOLLETE PLUME 24 NERO,114.99,68.99,Decollete donna,Stonefly,35.0,41.0
3,SCARPE CON TACCO IRIS 1 ORO,119.99,95.99,Decollete donna,Stonefly,35.0,41.0
4,SCARPE DECOLLETE PLUME 24 BLU SCURO,114.99,68.99,Decollete donna,Stonefly,35.0,41.0
5,SCARPE DECOLLETE PLUME 24 ORO,114.99,68.99,Decollete donna,Stonefly,35.0,41.0
6,SCARPE CON TACCO IRIS 7 BEIGE,119.99,71.99,Decollete donna,Stonefly,35.0,41.0
7,SCARPE CON TACCO BLANCA 3 NERO,109.99,65.99,Decollete donna,Stonefly,35.0,41.0
8,SCARPE DECOLLETE OXA 12 NERO,109.99,65.99,Decollete donna,Stonefly,35.0,41.0
9,SCARPE MARY JANE OXA 6 NERO,109.99,65.99,Decollete donna,Stonefly,35.0,41.0


In [5]:
#senaker
nomi = []
prezzi_scontati = []
categorie = []
brands = []
prezzi_iniziali = []
taglie = []

for pagina in range(1, 5):
    url = f"https://www.stonefly.it/collections/sneakers-donna?p={pagina}" 
    print("Elaboro pagina", pagina, "alla URL:", url)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Estrai categoria e brand una volta per pagina
    categoria_tag = soup.find("h1", class_="hyphenate")
    categoria = categoria_tag.text.strip() if categoria_tag else "N/A"
    
    brand_tag = soup.find("img", class_="header__logo-image")
    src = brand_tag.get("src", "")
    brand = src.split("/")[-1].split(".")[0].replace("Logo", "")

    # Cerca le taglie nei filtri della pagina
    taglie_divs = soup.find_all("label")  # o "input", "div" - dipende dalla struttura HTML
    taglie_pagina = []
    
    for div in taglie_divs:
        testo = div.get_text().strip()
        # Cerca numeri che sembrano taglie (es: 36, 37, 38.5, etc.)
        if re.match(r'^\d+(\.\d+)?$', testo):
            try:
                taglia = float(testo)
                if 30 <= taglia <= 50:  # Range ragionevole per scarpe donna
                    taglie_pagina.append(taglia)
            except:
                pass
    
    # Rimuovi duplicati
    taglie_pagina = list(set(taglie_pagina))
    taglie_numeriche = taglie_pagina if taglie_pagina else []
    
    if taglie_numeriche:
        taglia_min = min(taglie_numeriche)
        taglia_max = max(taglie_numeriche)
    else:
        taglia_min = None
        taglia_max = None
    
    prodotti = soup.find_all('div', class_='each-product-gtm')
    
    for prodotto in prodotti:
        nome = prodotto.find("a", class_="bold")
        nome = nome.text
        nomi.append(nome)
        
        prezzo_scontato = prodotto.find("sale-price", class_="text-on-sale")
        prezzo_scontato = prezzo_scontato.text
        prezzo_scontato = prezzo_scontato.replace("Prezzo scontato", "").replace("€", "").strip()
        prezzo_scontato = prezzo_scontato.replace(",", ".")
        prezzo_scontato = float(prezzo_scontato)
        prezzi_scontati.append(prezzo_scontato)
    
        prezzo_iniziale = prodotto.find("compare-at-price", class_="text-subdued line-through")
        prezzo_iniziale = prezzo_iniziale.text
        prezzo_iniziale = prezzo_iniziale.replace("Prezzo", "").replace("€", "").strip()
        prezzo_iniziale = prezzo_iniziale.replace(",", ".")
        prezzo_iniziale = float(prezzo_iniziale)
        prezzi_iniziali.append(prezzo_iniziale)
        
        
        categorie.append(categoria)
        brands.append(brand)
        taglie.append((taglia_min, taglia_max))

df_sne = pd.DataFrame({
    "nome": nomi,
    "prezzo_iniziale": prezzi_iniziali,
    "prezzo_scontato": prezzi_scontati,
    "categoria": categorie,
    "brand": brands,
    "taglia_min": [t[0] for t in taglie],
    "taglia_max": [t[1] for t in taglie] 
})
df_sne

Elaboro pagina 1 alla URL: https://www.stonefly.it/collections/sneakers-donna?p=1
Elaboro pagina 2 alla URL: https://www.stonefly.it/collections/sneakers-donna?p=2
Elaboro pagina 3 alla URL: https://www.stonefly.it/collections/sneakers-donna?p=3
Elaboro pagina 4 alla URL: https://www.stonefly.it/collections/sneakers-donna?p=4


,nome,prezzo_iniziale,prezzo_scontato,categoria,brand,taglia_min,taglia_max
0,SCARPE ALLACCIATE ELLA 12 BIANCO,129.99,65.00,Sneakers donna,Stonefly,35.0,43.0
1,SLIP-ON SHOES CREAM 38 GRIGIO CHIARO,119.99,95.99,Sneakers donna,Stonefly,35.0,43.0
2,SCARPE ALLACCIATE ELETTRA 8 BEIGE,109.99,55.00,Sneakers donna,Stonefly,35.0,43.0
3,SCARPE ALLACCIATE ELLA 3 BIANCO,124.99,62.50,Sneakers donna,Stonefly,35.0,43.0
4,SCARPE CON CERNIERA CREAM 52 GRIGIO CHIARO,124.99,99.99,Sneakers donna,Stonefly,35.0,43.0
...,...,...,...,...,...,...,...
187,SLIP-ON SHOES CREAM 21 BLU,119.99,60.00,Sneakers donna,Stonefly,35.0,43.0
188,SCARPE CON CERNIERA CREAM 52 GRIGIO CHIARO,124.99,87.49,Sneakers donna,Stonefly,35.0,43.0
189,SCARPE ALLACCIATE TWINS 3 NERO,119.99,71.99,Sneakers donna,Stonefly,35.0,43.0
190,SCARPE ALLACCIATE TWIGGY 1 BIANCO,119.99,71.99,Sneakers donna,Stonefly,35.0,43.0


In [6]:
nomi = []
prezzi_scontati = []
categorie = []
brands = []
prezzi_iniziali = []
taglie = []

for pagina in range(1, 3):
    url = f"https://www.stonefly.it/collections/mocassini-donna?p={pagina}"
    print("Elaboro pagina", pagina, "alla URL:", url)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Estrai categoria e brand una volta per pagina
    categoria_tag = soup.find("h1", class_="hyphenate")
    categoria = categoria_tag.text.strip() if categoria_tag else "N/A"
    
    brand_tag = soup.find("img", class_="header__logo-image")
    src = brand_tag.get("src", "")
    brand = src.split("/")[-1].split(".")[0].replace("Logo", "")

    for div in taglie_divs:
        testo = div.get_text().strip()
        # Cerca numeri che sembrano taglie (es: 36, 37, 38.5, etc.)
        if re.match(r'^\d+(\.\d+)?$', testo):
            try:
                taglia = float(testo)
                if 30 <= taglia <= 50:  # Range ragionevole per scarpe donna
                    taglie_pagina.append(taglia)
            except:
                pass
    
    # Rimuovi duplicati
    taglie_pagina = list(set(taglie_pagina))
    taglie_numeriche = taglie_pagina if taglie_pagina else []
    
    if taglie_numeriche:
        taglia_min = min(taglie_numeriche)
        taglia_max = max(taglie_numeriche)
    else:
        taglia_min = None
        taglia_max = None
    
    
    prodotti = soup.find_all('div', class_='each-product-gtm')
    
    for prodotto in prodotti:
        # Controlla prima se il prodotto è in saldo
        prezzo_scontato_tag = prodotto.find("sale-price", class_="text-on-sale")
        prezzo_iniziale_tag = prodotto.find("compare-at-price", class_="text-subdued line-through")
        
        # Se non ha prezzo scontato O prezzo iniziale, salta il prodotto
        if not prezzo_scontato_tag or not prezzo_iniziale_tag:
            print("Prodotto non in saldo - saltato")
            continue
        
        # Se arriviamo qui, il prodotto è in saldo
        nome = prodotto.find("a", class_="bold")
        nome = nome.text
        nomi.append(nome)
        
        prezzo_scontato = prezzo_scontato_tag.text
        prezzo_scontato = prezzo_scontato.replace("Prezzo scontato", "").replace("€", "").strip()
        prezzo_scontato = prezzo_scontato.replace(",", ".")
        prezzo_scontato = float(prezzo_scontato)
        prezzi_scontati.append(prezzo_scontato)
    
        prezzo_iniziale = prezzo_iniziale_tag.text
        prezzo_iniziale = prezzo_iniziale.replace("Prezzo", "").replace("€", "").strip()
        prezzo_iniziale = prezzo_iniziale.replace(",", ".")
        prezzo_iniziale = float(prezzo_iniziale)
        prezzi_iniziali.append(prezzo_iniziale)
        
        categorie.append(categoria)
        brands.append(brand)
        taglie.append((taglia_min, taglia_max))
        

df_moc = pd.DataFrame({
    "nome": nomi,
    "prezzo_iniziale": prezzi_iniziali,
    "prezzo_scontato": prezzi_scontati,
    "categoria": categorie,
    "brand": brands,
    "taglia_min": [t[0] for t in taglie],
    "taglia_max": [t[1] for t in taglie]
})
df_moc

Elaboro pagina 1 alla URL: https://www.stonefly.it/collections/mocassini-donna?p=1
Prodotto non in saldo - saltato
Elaboro pagina 2 alla URL: https://www.stonefly.it/collections/mocassini-donna?p=2
Prodotto non in saldo - saltato


,nome,prezzo_iniziale,prezzo_scontato,categoria,brand,taglia_min,taglia_max
0,MOCASSINI CAPRI 1 BLU SCURO,99.99,59.99,Mocassini donna,Stonefly,35.0,43.0
1,MOCASSINI CAPRI 1 BIANCO,109.99,65.99,Mocassini donna,Stonefly,35.0,43.0
2,MOCASSINI PASEO IV 1 GRIGIO CHIARO,109.99,87.99,Mocassini donna,Stonefly,35.0,43.0
3,MOCASSINI PASEO IV 1 NERO,109.99,87.99,Mocassini donna,Stonefly,35.0,43.0
4,MOCASSINI CAPRI 1 ROSSO,99.99,59.99,Mocassini donna,Stonefly,35.0,43.0
...,...,...,...,...,...,...,...
89,MOCASSINI PLUME 13 GRIGIO SCURO,119.99,60.00,Mocassini donna,Stonefly,35.0,43.0
90,MOCASSINI PHOEBE 16 VIOLA CHIARO,109.99,55.00,Mocassini donna,Stonefly,35.0,43.0
91,MOCASSINI CELINE 5 BEIGE,109.99,65.99,Mocassini donna,Stonefly,35.0,43.0
92,MOCASSINI CELINE 3 BIANCO,114.99,91.99,Mocassini donna,Stonefly,35.0,43.0


In [7]:
#inizio sandali
nomi = []
prezzi_scontati = []
categorie = []
brands = []
prezzi_iniziali = []
taglie = []

for pagina in range(1, 3):
    url = f"https://www.stonefly.it/collections/sandali-donna?p={pagina}"  # Aggiunta f
    print("Elaboro pagina", pagina, "alla URL:", url)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Estrai categoria e brand una volta per pagina
    categoria_tag = soup.find("h1", class_="hyphenate")
    categoria = categoria_tag.text.strip() if categoria_tag else "N/A"
    
    brand_tag = soup.find("img", class_="header__logo-image")
    src = brand_tag.get("src", "")
    brand = src.split("/")[-1].split(".")[0].replace("Logo", "")

    for div in taglie_divs:
        testo = div.get_text().strip()
        # Cerca numeri che sembrano taglie (es: 36, 37, 38.5, etc.)
        if re.match(r'^\d+(\.\d+)?$', testo):
            try:
                taglia = float(testo)
                if 30 <= taglia <= 50:  # Range ragionevole per scarpe donna
                    taglie_pagina.append(taglia)
            except:
                pass
    
    # Rimuovi duplicati
    taglie_pagina = list(set(taglie_pagina))
    taglie_numeriche = taglie_pagina if taglie_pagina else []
    
    if taglie_numeriche:
        taglia_min = min(taglie_numeriche)
        taglia_max = max(taglie_numeriche)
    else:
        taglia_min = None
        taglia_max = None
    
    prodotti = soup.find_all('div', class_='each-product-gtm')
    
    for prodotto in prodotti:
        # Controlla prima se il prodotto è in saldo
        prezzo_scontato_tag = prodotto.find("sale-price", class_="text-on-sale")
        prezzo_iniziale_tag = prodotto.find("compare-at-price", class_="text-subdued line-through")
        
        # Se non ha prezzo scontato O prezzo iniziale, salta il prodotto
        if not prezzo_scontato_tag or not prezzo_iniziale_tag:
            nome_saltato = prodotto.find("a", class_="bold")
            nome_saltato = nome_saltato.text if nome_saltato else "Nome non trovato"
            print(f"Prodotto '{nome_saltato}' non in saldo - saltato")
            continue
        
        nome = prodotto.find("a", class_="bold")
        nome = nome.text
        nomi.append(nome)
        
        # Gestisci il prezzo scontato
        prezzo_scontato = prezzo_scontato_tag.text
        prezzo_scontato = prezzo_scontato.replace("Prezzo scontato", "").replace("€", "").strip()
        
        # Gestisci "A partire da" e altri testi
        if "A partire da" in prezzo_scontato:
            prezzo_scontato = prezzo_scontato.replace("A partire da", "").strip()
        
        prezzo_scontato = prezzo_scontato.replace(",", ".")
        prezzo_scontato = float(prezzo_scontato)
        prezzi_scontati.append(prezzo_scontato)
        
        # Gestisci il prezzo iniziale
        prezzo_iniziale = prezzo_iniziale_tag.text
        prezzo_iniziale = prezzo_iniziale.replace("Prezzo", "").replace("€", "").strip()
        
        # Gestisci "A partire da" anche per il prezzo iniziale
        if "A partire da" in prezzo_iniziale:
            prezzo_iniziale = prezzo_iniziale.replace("A partire da", "").strip()
        
        prezzo_iniziale = prezzo_iniziale.replace(",", ".")
        prezzo_iniziale = float(prezzo_iniziale)
        prezzi_iniziali.append(prezzo_iniziale)
        
        categorie.append(categoria)
        brands.append(brand)
        taglie.append((taglia_min, taglia_max))

df_san1 = pd.DataFrame({
    "nome": nomi,
    "prezzo_iniziale": prezzi_iniziali,
    "prezzo_scontato": prezzi_scontati,
    "categoria": "Sandali",
    "tipologia": "Sandali con tacco comodo",
    "brand": brands 
})
df_san1

Elaboro pagina 1 alla URL: https://www.stonefly.it/collections/sandali-donna?p=1
Elaboro pagina 2 alla URL: https://www.stonefly.it/collections/sandali-donna?p=2


,nome,prezzo_iniziale,prezzo_scontato,categoria,tipologia,brand
0,SANDALI ELODY 24 VERDE CHIARO,114.99,91.99,Sandali,Sandali con tacco comodo,Stonefly
1,SANDALI PLUME 28 ORO,99.99,59.99,Sandali,Sandali con tacco comodo,Stonefly
2,SANDALI ONDA 6 BIANCO,99.99,59.99,Sandali,Sandali con tacco comodo,Stonefly
3,SANDALI ELODY 24 BIANCO,114.99,91.99,Sandali,Sandali con tacco comodo,Stonefly
4,SANDALI BASSI PARKY 15 BLU,99.99,59.99,Sandali,Sandali con tacco comodo,Stonefly
...,...,...,...,...,...,...
91,SANDALI ONDA 4 NERO,109.99,65.99,Sandali,Sandali con tacco comodo,Stonefly
92,SANDALI BASSI AQUA 11 BLU SCURO,99.99,59.99,Sandali,Sandali con tacco comodo,Stonefly
93,SANDALI PLUME 34 ORO,99.99,79.99,Sandali,Sandali con tacco comodo,Stonefly
94,SANDALI ELODY 23 ROSA,99.99,69.99,Sandali,Sandali con tacco comodo,Stonefly


In [8]:


# Continuano i sandali
nomi = []
prezzi_scontati = []
categorie = []
brands = []
prezzi_iniziali = []
taglie = []

url = "https://www.stonefly.it/collections/sandali-tacco-donna"
print("Elaboro pagina alla URL:", url)
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Estrai categoria e brand una volta per pagina
categoria_tag = soup.find("h1", class_="hyphenate")
categoria = categoria_tag.text.strip() if categoria_tag else "N/A"

brand_tag = soup.find("img", class_="header__logo-image")
src = brand_tag.get("src", "") if brand_tag else ""
brand = src.split("/")[-1].split(".")[0].replace("Logo", "") if src else "N/A"

# Estrazione taglie
taglie_divs = soup.find_all("label")  # o "input", "div" - dipende dalla struttura HTML
taglie_pagina = []

for div in taglie_divs:
    testo = div.get_text().strip()
    # Cerca numeri che sembrano taglie (es: 36, 37, 38.5, etc.)
    if re.match(r'^\d+(\.\d+)?$', testo):
        try:
            taglia = float(testo)
            if 30 <= taglia <= 50:  # Range ragionevole per scarpe donna
                taglie_pagina.append(taglia)
        except:
            pass

# Rimuovi duplicati
taglie_pagina = list(set(taglie_pagina))
taglie_numeriche = taglie_pagina if taglie_pagina else []

if taglie_numeriche:
    taglia_min = min(taglie_numeriche)
    taglia_max = max(taglie_numeriche)
else:
    taglia_min = None
    taglia_max = None

# Estrazione prodotti
prodotti = soup.find_all('div', class_='each-product-gtm')

for prodotto in prodotti:
    # Controlla prima se il prodotto è in saldo
    prezzo_scontato_tag = prodotto.find("sale-price", class_="text-on-sale")
    prezzo_iniziale_tag = prodotto.find("compare-at-price", class_="text-subdued line-through")
    
    # Se non ha prezzo scontato O prezzo iniziale, salta il prodotto
    if not prezzo_scontato_tag or not prezzo_iniziale_tag:
        nome_saltato = prodotto.find("a", class_="bold")
        nome_saltato = nome_saltato.text if nome_saltato else "Nome non trovato"
        print(f"Prodotto '{nome_saltato}' non in saldo - saltato")
        continue
    
    nome = prodotto.find("a", class_="bold")
    nome = nome.text
    nomi.append(nome)
    
    # Gestisci il prezzo scontato
    prezzo_scontato = prezzo_scontato_tag.text
    prezzo_scontato = prezzo_scontato.replace("Prezzo scontato", "").replace("€", "").strip()
    
    # Gestisci "A partire da" e altri testi
    if "A partire da" in prezzo_scontato:
        prezzo_scontato = prezzo_scontato.replace("A partire da", "").strip()
    
    prezzo_scontato = prezzo_scontato.replace(",", ".")
    prezzo_scontato = float(prezzo_scontato)
    prezzi_scontati.append(prezzo_scontato)
    
    # Gestisci il prezzo iniziale
    prezzo_iniziale = prezzo_iniziale_tag.text
    prezzo_iniziale = prezzo_iniziale.replace("Prezzo", "").replace("€", "").strip()
    
    # Gestisci "A partire da" anche per il prezzo iniziale
    if "A partire da" in prezzo_iniziale:
        prezzo_iniziale = prezzo_iniziale.replace("A partire da", "").strip()
    
    prezzo_iniziale = prezzo_iniziale.replace(",", ".")
    prezzo_iniziale = float(prezzo_iniziale)
    prezzi_iniziali.append(prezzo_iniziale)
    
    categorie.append(categoria)
    brands.append(brand)
    taglie.append((taglia_min, taglia_max))

# Creazione DataFrame
df_san2 = pd.DataFrame({
    "nome": nomi,
    "prezzo_iniziale": prezzi_iniziali,
    "prezzo_scontato": prezzi_scontati,
    "categoria": "Sandali",
    "tipologia": "Sandali con tacco alto",
    "brand": brands,
    "taglia_min": [t[0] for t in taglie],
    "taglia_max": [t[1] for t in taglie]
})

df_san2

Elaboro pagina alla URL: https://www.stonefly.it/collections/sandali-tacco-donna


,nome,prezzo_iniziale,prezzo_scontato,categoria,tipologia,brand,taglia_min,taglia_max
0,SANDALI CON TACCO THALIA 2 BEIGE,119.99,83.99,Sandali,Sandali con tacco alto,Stonefly,35.0,41.0
1,SANDALI CON TACCO JENNY 6 GIALLO SCURO,109.99,65.99,Sandali,Sandali con tacco alto,Stonefly,35.0,41.0
2,SANDALI CON TACCO THALIA 2 BLU SCURO,119.99,83.99,Sandali,Sandali con tacco alto,Stonefly,35.0,41.0
3,SANDALI CON TACCO KARA 4 BIANCO,114.99,80.49,Sandali,Sandali con tacco alto,Stonefly,35.0,41.0
4,SANDALI CON TACCO JENNY 7 NERO,109.99,65.99,Sandali,Sandali con tacco alto,Stonefly,35.0,41.0
5,SANDALI CON TACCO THALIA 2 ROSA SCURO,119.99,83.99,Sandali,Sandali con tacco alto,Stonefly,35.0,41.0
6,SANDALI CON TACCO KARA 4 NERO,114.99,80.49,Sandali,Sandali con tacco alto,Stonefly,35.0,41.0
7,SANDALI CON TACCO KARA 1 ARGENTO,109.99,76.99,Sandali,Sandali con tacco alto,Stonefly,35.0,41.0
8,SANDALI CON TACCO JENNY 7 GRIGIO CHIARO,109.99,65.99,Sandali,Sandali con tacco alto,Stonefly,35.0,41.0
9,SANDALI CON TACCO TALITHA 2 NERO,119.99,71.99,Sandali,Sandali con tacco alto,Stonefly,35.0,41.0


In [9]:
#finisco i sandali
nomi = []
prezzi_scontati = []
categorie = []
brands = []
prezzi_iniziali = []
taglie = []

for pagina in range(1, 5):
    url = "https://www.stonefly.it/collections/sandali-zeppa-donna?p={pagina}"
    print("Elaboro pagina", pagina, "alla URL:", url)
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Estrai categoria e brand una volta per pagina
    categoria_tag = soup.find("h1", class_="hyphenate")
    categoria = categoria_tag.text.strip() if categoria_tag else "N/A"
    
    brand_tag = soup.find("img", class_="header__logo-image")
    src = brand_tag.get("src", "")
    brand = src.split("/")[-1].split(".")[0].replace("Logo", "")

    for div in taglie_divs:
        testo = div.get_text().strip()
        # Cerca numeri che sembrano taglie (es: 36, 37, 38.5, etc.)
        if re.match(r'^\d+(\.\d+)?$', testo):
            try:
                taglia = float(testo)
                if 30 <= taglia <= 50:  # Range ragionevole per scarpe donna
                    taglie_pagina.append(taglia)
            except:
                pass
    
    # Rimuovi duplicati
    taglie_pagina = list(set(taglie_pagina))
    taglie_numeriche = taglie_pagina if taglie_pagina else []
    
    if taglie_numeriche:
        taglia_min = min(taglie_numeriche)
        taglia_max = max(taglie_numeriche)
    else:
        taglia_min = None
        taglia_max = None
    
    prodotti = soup.find_all('div', class_='each-product-gtm')
    
    for prodotto in prodotti:
        # Controlla prima se il prodotto è in saldo
        prezzo_scontato_tag = prodotto.find("sale-price", class_="text-on-sale")
        prezzo_iniziale_tag = prodotto.find("compare-at-price", class_="text-subdued line-through")
        
        # Se non ha prezzo scontato O prezzo iniziale, salta il prodotto
        if not prezzo_scontato_tag or not prezzo_iniziale_tag:
            nome_saltato = prodotto.find("a", class_="bold")
            nome_saltato = nome_saltato.text if nome_saltato else "Nome non trovato"
            print(f"Prodotto '{nome_saltato}' non in saldo - saltato")
            continue
        
        nome = prodotto.find("a", class_="bold")
        nome = nome.text
        nomi.append(nome)
        
        # Gestisci il prezzo scontato
        prezzo_scontato = prezzo_scontato_tag.text
        prezzo_scontato = prezzo_scontato.replace("Prezzo scontato", "").replace("€", "").strip()
        
        # Gestisci "A partire da" e altri testi
        if "A partire da" in prezzo_scontato:
            prezzo_scontato = prezzo_scontato.replace("A partire da", "").strip()
        
        prezzo_scontato = prezzo_scontato.replace(",", ".")
        prezzo_scontato = float(prezzo_scontato)
        prezzi_scontati.append(prezzo_scontato)
        
        # Gestisci il prezzo iniziale
        prezzo_iniziale = prezzo_iniziale_tag.text
        prezzo_iniziale = prezzo_iniziale.replace("Prezzo", "").replace("€", "").strip()
        
        # Gestisci "A partire da" anche per il prezzo iniziale
        if "A partire da" in prezzo_iniziale:
            prezzo_iniziale = prezzo_iniziale.replace("A partire da", "").strip()
        
        prezzo_iniziale = prezzo_iniziale.replace(",", ".")
        prezzo_iniziale = float(prezzo_iniziale)
        prezzi_iniziali.append(prezzo_iniziale)
        
        categorie.append(categoria)
        brands.append(brand)
        taglie.append((taglia_min, taglia_max))

df_san3 = pd.DataFrame({
    "nome": nomi,
    "prezzo_iniziale": prezzi_iniziali,
    "prezzo_scontato": prezzi_scontati,
    "categoria": "Sandali",
    "tipologia": "Sandali con zeppa",
    "brand": brands 
})
df_san3

Elaboro pagina 1 alla URL: https://www.stonefly.it/collections/sandali-zeppa-donna?p={pagina}
Elaboro pagina 2 alla URL: https://www.stonefly.it/collections/sandali-zeppa-donna?p={pagina}
Elaboro pagina 3 alla URL: https://www.stonefly.it/collections/sandali-zeppa-donna?p={pagina}
Elaboro pagina 4 alla URL: https://www.stonefly.it/collections/sandali-zeppa-donna?p={pagina}


,nome,prezzo_iniziale,prezzo_scontato,categoria,tipologia,brand
0,SANDALI CON ZEPPA ELODY 9 BIANCO,99.99,59.99,Sandali,Sandali con zeppa,Stonefly
1,SANDALI CON ZEPPA RIVA 6 BEIGE,99.99,59.99,Sandali,Sandali con zeppa,Stonefly
2,SANDALI CON ZEPPA PARKY 24 ORO,119.99,95.99,Sandali,Sandali con zeppa,Stonefly
3,SANDALI CON ZEPPA AQUA 2 ORO,99.99,79.99,Sandali,Sandali con zeppa,Stonefly
4,SANDALI CON ZEPPA AQUA 2 BLU SCURO,99.99,69.99,Sandali,Sandali con zeppa,Stonefly
...,...,...,...,...,...,...
187,SANDALI CON ZEPPA PARKY 27 NERO,109.99,76.99,Sandali,Sandali con zeppa,Stonefly
188,SANDALI CON ZEPPA PARKY 27 ORO,109.99,87.99,Sandali,Sandali con zeppa,Stonefly
189,SANDALI CON ZEPPA ELY 20 BEIGE,109.99,65.99,Sandali,Sandali con zeppa,Stonefly
190,SANDALI CON ZEPPA ELODY 20 NERO,109.99,65.99,Sandali,Sandali con zeppa,Stonefly


In [10]:
df_san = pd.concat([df_san1, df_san2, df_san3])
df_san

,nome,prezzo_iniziale,prezzo_scontato,categoria,tipologia,brand,taglia_min,taglia_max
0,SANDALI ELODY 24 VERDE CHIARO,114.99,91.99,Sandali,Sandali con tacco comodo,Stonefly,NaN,NaN
1,SANDALI PLUME 28 ORO,99.99,59.99,Sandali,Sandali con tacco comodo,Stonefly,NaN,NaN
2,SANDALI ONDA 6 BIANCO,99.99,59.99,Sandali,Sandali con tacco comodo,Stonefly,NaN,NaN
3,SANDALI ELODY 24 BIANCO,114.99,91.99,Sandali,Sandali con tacco comodo,Stonefly,NaN,NaN
4,SANDALI BASSI PARKY 15 BLU,99.99,59.99,Sandali,Sandali con tacco comodo,Stonefly,NaN,NaN
...,...,...,...,...,...,...,...,...
187,SANDALI CON ZEPPA PARKY 27 NERO,109.99,76.99,Sandali,Sandali con zeppa,Stonefly,NaN,NaN
188,SANDALI CON ZEPPA PARKY 27 ORO,109.99,87.99,Sandali,Sandali con zeppa,Stonefly,NaN,NaN
189,SANDALI CON ZEPPA ELY 20 BEIGE,109.99,65.99,Sandali,Sandali con zeppa,Stonefly,NaN,NaN
190,SANDALI CON ZEPPA ELODY 20 NERO,109.99,65.99,Sandali,Sandali con zeppa,Stonefly,NaN,NaN


In [11]:
df_stonefly = pd.concat([df_san, df_sne, df_dec, df_moc], ignore_index=True)
df_stonefly

,nome,prezzo_iniziale,prezzo_scontato,categoria,tipologia,brand,taglia_min,taglia_max
0,SANDALI ELODY 24 VERDE CHIARO,114.99,91.99,Sandali,Sandali con tacco comodo,Stonefly,NaN,NaN
1,SANDALI PLUME 28 ORO,99.99,59.99,Sandali,Sandali con tacco comodo,Stonefly,NaN,NaN
2,SANDALI ONDA 6 BIANCO,99.99,59.99,Sandali,Sandali con tacco comodo,Stonefly,NaN,NaN
3,SANDALI ELODY 24 BIANCO,114.99,91.99,Sandali,Sandali con tacco comodo,Stonefly,NaN,NaN
4,SANDALI BASSI PARKY 15 BLU,99.99,59.99,Sandali,Sandali con tacco comodo,Stonefly,NaN,NaN
...,...,...,...,...,...,...,...,...
615,MOCASSINI PLUME 13 GRIGIO SCURO,119.99,60.00,Mocassini donna,NaN,Stonefly,35.0,43.0
616,MOCASSINI PHOEBE 16 VIOLA CHIARO,109.99,55.00,Mocassini donna,NaN,Stonefly,35.0,43.0
617,MOCASSINI CELINE 5 BEIGE,109.99,65.99,Mocassini donna,NaN,Stonefly,35.0,43.0
618,MOCASSINI CELINE 3 BIANCO,114.99,91.99,Mocassini donna,NaN,Stonefly,35.0,43.0


In [12]:
#proviamo ad estrarre i colori dai nomi dei prodotti
colori = ["NERO", "BIANCO", "BLU SCURO","GIALLO SCURO","VERDE SCURO","MARRONE", "ARGENTO", "ROSSO","BLU", "GRIGIO CHIARO", "VERDE CHIARO", "VIOLA CHIARO", "BEIGE", "ROSA", "GRIGIO", "AZZURRO", "ORO"]
colori_trovati = []  # lista per salvare i colori trovati

for nome in df_stonefly["nome"]:
    colore_trovato = "sconosciuto"  # valore di default
    for colore in colori:
        if colore in nome:
            colore_trovato = colore
          
    colori_trovati.append(colore_trovato)

df_stonefly["colore"] = colori_trovati
df_stonefly

,nome,prezzo_iniziale,prezzo_scontato,categoria,tipologia,brand,taglia_min,taglia_max,colore
0,SANDALI ELODY 24 VERDE CHIARO,114.99,91.99,Sandali,Sandali con tacco comodo,Stonefly,NaN,NaN,VERDE CHIARO
1,SANDALI PLUME 28 ORO,99.99,59.99,Sandali,Sandali con tacco comodo,Stonefly,NaN,NaN,ORO
2,SANDALI ONDA 6 BIANCO,99.99,59.99,Sandali,Sandali con tacco comodo,Stonefly,NaN,NaN,BIANCO
3,SANDALI ELODY 24 BIANCO,114.99,91.99,Sandali,Sandali con tacco comodo,Stonefly,NaN,NaN,BIANCO
4,SANDALI BASSI PARKY 15 BLU,99.99,59.99,Sandali,Sandali con tacco comodo,Stonefly,NaN,NaN,BLU
...,...,...,...,...,...,...,...,...,...
615,MOCASSINI PLUME 13 GRIGIO SCURO,119.99,60.00,Mocassini donna,NaN,Stonefly,35.0,43.0,GRIGIO
616,MOCASSINI PHOEBE 16 VIOLA CHIARO,109.99,55.00,Mocassini donna,NaN,Stonefly,35.0,43.0,VIOLA CHIARO
617,MOCASSINI CELINE 5 BEIGE,109.99,65.99,Mocassini donna,NaN,Stonefly,35.0,43.0,BEIGE
618,MOCASSINI CELINE 3 BIANCO,114.99,91.99,Mocassini donna,NaN,Stonefly,35.0,43.0,BIANCO


In [13]:
filtro_c= df_stonefly["colore"]=="sconosciuto"
print(df_stonefly.loc[filtro_c,"nome"])

Series([], Name: nome, dtype: object)


In [14]:
df = df_stonefly

In [15]:
df["sconto (%)"] = round(((df["prezzo_iniziale"] - df["prezzo_scontato"]) / df["prezzo_iniziale"])*100, 2)
df

,nome,prezzo_iniziale,prezzo_scontato,categoria,tipologia,brand,taglia_min,taglia_max,colore,sconto (%)
0,SANDALI ELODY 24 VERDE CHIARO,114.99,91.99,Sandali,Sandali con tacco comodo,Stonefly,NaN,NaN,VERDE CHIARO,20.0
1,SANDALI PLUME 28 ORO,99.99,59.99,Sandali,Sandali con tacco comodo,Stonefly,NaN,NaN,ORO,40.0
2,SANDALI ONDA 6 BIANCO,99.99,59.99,Sandali,Sandali con tacco comodo,Stonefly,NaN,NaN,BIANCO,40.0
3,SANDALI ELODY 24 BIANCO,114.99,91.99,Sandali,Sandali con tacco comodo,Stonefly,NaN,NaN,BIANCO,20.0
4,SANDALI BASSI PARKY 15 BLU,99.99,59.99,Sandali,Sandali con tacco comodo,Stonefly,NaN,NaN,BLU,40.0
...,...,...,...,...,...,...,...,...,...,...
615,MOCASSINI PLUME 13 GRIGIO SCURO,119.99,60.00,Mocassini donna,NaN,Stonefly,35.0,43.0,GRIGIO,50.0
616,MOCASSINI PHOEBE 16 VIOLA CHIARO,109.99,55.00,Mocassini donna,NaN,Stonefly,35.0,43.0,VIOLA CHIARO,50.0
617,MOCASSINI CELINE 5 BEIGE,109.99,65.99,Mocassini donna,NaN,Stonefly,35.0,43.0,BEIGE,40.0
618,MOCASSINI CELINE 3 BIANCO,114.99,91.99,Mocassini donna,NaN,Stonefly,35.0,43.0,BIANCO,20.0


In [17]:
df["tipologia"].unique()

array(['Sandali con tacco comodo', 'Sandali con tacco alto',
       'Sandali con zeppa', nan], dtype=object)

In [21]:
df["categoria"] = df["categoria"].replace("Decollete donna", "Decollete") 
df["categoria"] = df["categoria"].replace("Sneakers donna", "Sneakers") 
df["categoria"] = df["categoria"].replace("Mocassini donna", "Mocassini") 

In [28]:
df["categoria"].unique()

array(['Sandali', 'Sneakers', 'Decollete', 'Mocassini'], dtype=object)

In [29]:
#df.to_csv('Stonefly_sconto_size.csv', index=False)